In [ ]:
!pip install -q facenet-pytorch
!pip install pillow==6.2.2

In [ ]:
import pandas as pd
from fastai.vision.all import *
from fastai.torch_basics import *
from PIL import Image
import os
from facenet_pytorch import InceptionResnetV1
os.listdir('/kaggle/input/identity-employees-in-surveillance-cctv/dataset/dataset/train')

In [ ]:
path = '/kaggle/input/identity-employees-in-surveillance-cctv/dataset/dataset/train/'
images_path= path + 'images/'
test_path = '/kaggle/input/identity-employees-in-surveillance-cctv/dataset_unseen/unseen_test/images/'
csv_path = path + 'labels.csv'
csv_path

In [ ]:
files = get_image_files(images_path)
df = pd.read_csv(csv_path)
df.head()

In [ ]:
# Create ImageDataLoaders from CSV
dls = ImageDataLoaders.from_csv(
    path='/kaggle/input/identity-employees-in-surveillance-cctv/dataset/dataset/train/',
    csv_fname='labels.csv',
    folder='images',
    fn_col='filename',
    label_col='emp_id',
    valid_pct=0.2,
    item_tfms=Resize(224),
    batch_tfms=aug_transforms(size=224)
)
# Show batch to verify the data loading
dls.show_batch()
df.groupby('emp_id').size()

In [ ]:
from torch import nn
from facenet_pytorch import InceptionResnetV1

# Load FaceNet backbone
facenet_backbone = InceptionResnetV1(pretrained='vggface2')  # can use 'casia-webface' too

# Freeze the backbone if needed
facenet_backbone.classify = False
facenet_backbone.eval()  # optional: only if you're not training

# Wrap into a model with a classification head
num_classes = len(dls.vocab)  # number of emp_id classes
model = nn.Sequential(
    facenet_backbone,
    nn.Linear(512, num_classes)
)

# Use FastAI learner
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, model_dir='/kaggle/working/models')
learn.fine_tune(175)  # 👈 Train for 180 epochs


# save weights

In [ ]:
learn.save('/kaggle/working/facenet_weights')

In [ ]:
def predict_image(image):
	return learn.predict(test_path+image)[0]
#test_path
results = []

In [ ]:
for image in os.listdir(test_path):
	prediction = dict(filename=image, emp_id=predict_image(image))
	results.append(prediction)

In [ ]:
results_df = pd.DataFrame.from_dict(results)
hi = results_df.drop_duplicates()
hi

In [ ]:
hi.to_csv('/kaggle/working/submission_maybe_maybe?.csv', index=False)